# Generate the datasets
This program is used to generate the data, including test.csv, train.csv, valid.csv.

## import package

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

## parameter

In [2]:
data_number = 10000

# 定義資料數量、訓練集、驗證集和測試集的比例
train_p, valid_p, test_p = 0.7, 0.2, 0.1

## define the kinematics function

In [3]:
# 設定機械手臂的幾何參數
L1 = 0.    # a1 = 0
L2 = 135.  # a2 = 135
L3 = 147.  # a3 = 147
L4 = 61.   # a4 = 61

# 正向運動學方程式
def forward_kinematics_equations(theta):
    C1 = np.cos(theta[:,0])
    C2 = np.cos(theta[:,1])
    C23 = np.cos(theta[:,1] + theta[:,2])
    S1 = np.sin(theta[:,0])
    S2 = np.sin(theta[:,1])
    S23 = np.sin(theta[:,1] + theta[:,2])

    x = C1 * (L3 * C23 + L2 * C2 + L4)
    y = S1 * (L3 * C23 + L2 * C2 + L4)
    z = -L2 * S2 - L3 * S23 

    return np.vstack((x, y, z)).T

## generate the random thetas(joints)

In [4]:
thetas = np.random.uniform(-np.pi/2, np.pi/2, size=(data_number, 3))
# print(thetas)

## thetas to positions

In [5]:
points = forward_kinematics_equations(thetas)
datasets = np.hstack((points, thetas))
# print(datasets[:2,:]) # x, y, z, t1, t2, t3

In [6]:
# 計算資料集中各個部分的數量
train_num = int(data_number * train_p)
valid_num = int(data_number * valid_p)
test_num = data_number - train_num - valid_num
train = datasets[:train_num, :]
valid = datasets[train_num:train_num+valid_num, :]
test  = datasets[train_num+valid_num:data_number, :]
print(train.shape, valid.shape, test.shape)


(7000, 6) (2000, 6) (1000, 6)


## save datasets

In [7]:
# 創建data文件夾（如果不存在）
os.makedirs('data', exist_ok=True)

In [8]:
# 將資料寫入檔案
np.savetxt('data/train.csv', train, delimiter=',')
np.savetxt('data/valid.csv', valid, delimiter=',')
np.savetxt('data/test.csv', test, delimiter=',')